# <center> Workshop 3: Extreme Precipitation</center>

## Introduction

In this session we will answer a "real life" question regarding climate risk under warming: how may the frequency of surface flooding around the LU campus, driven by extreme precipitation, change as the climate warms? 

To address this, we will employ climate model projections from CMIP6, processing simulations from different models and experiments to generate the type of warming level plot discussed in the lecture. You should work together in the task, teaming up to work through the notebook, and coordinating between teams to ensure that the [shared spreadsheet](https://docs.google.com/spreadsheets/d/1SGONp3xEh62SLVh_fbkTSiarNls5ArcxXieAt51qvjw/edit?usp=sharing) is filled in as efficiently as possible. 


## Getting started

This is the most ambitious bit of coding we have tried so far. For it to work, it is essential that your laptop is setup correctly. To be reading this, you must have already installed Anaconda/Jupyter successfully. That is the critical first step, but you also need to have "xarray", "pandas" and "cdsapi" installed. If you haven't already done so, please install them now by opening the Terminal (macOS), or the Anaconda command prompt (Windows) and typing the following (press enter after each entry, and answer "yes" to any prompts"): 

conda install -c anaconda xarray
conda install -c conda-forge cdsapi

You *must* also have registered for a Copernicus account (as discussed in class). If you haven't done so yet register [here](https://cds.climate.copernicus.eu/user/register?destination=%2F%23!%2Fhome) now. 

Finally, your cdsapi key must be saved in your home directory. Follow [these instructions](https://cds.climate.copernicus.eu/api-how-to) (noting the separate link for Windows users; if you have a mac, you are counted as "Linux/Unix" user).

When you have completed the above, run the code below to get started! Errors at this point will likely reflect that something is not setup correctly. 

In [52]:
import os, cdsapi
import zipfile
import xarray as xa
import numpy as np, pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path
wd=str(Path().absolute())
folder_store="/".join(wd.split("/")[:-1])+"/Dev/CMIP6Data/"
if not os.path.isdir(folder_store):os.makedirs(folder_store)
%matplotlib inline
seconds_in_day=60**2*24
pid=0.68
c = cdsapi.Client()

If it ran OK, great! You can proceed; otherwise, ask Tom. 

In the below, we set various "global" options for our analysis. We specify that we're interested in exceedances of the 15 mm/day threshold (enough to cause surface flooding in the *winter*), and we also specify the latitude and logitude of the GYE weather station (lat_lough, and lon_lough). The other settings identify the months of the year we're interested in (months_sel: the months in the winter haf year) and the number of days in that period (ndays_sel). The latter enables us to use the relative frequency of exceedance to compute the expected number of flood days in a typical winter. 

Run the code below to set things up for analysis. 

In [53]:
# * * * * * * * * * * * * * * * * * * * #
# Setting  options/parameters
# * * * * * * * * * * * * * * * * * * * #
thresh=15 # mm/d after which flooding occurs (this is the winter threshold)
lat_lough=52.76 # Latitude
lon_lough=-1.23 # Longitude
month_sel=[1,2,3,10,11,12] # Months that we'll use (this is setup for winter)
ndays_year=182 # Number of days in Oct-March (i.e., winter)
# * * * * * * * * * * * * * * * * * * * #

## The observed flood frequency

We will now begin crunching some numbers -- starting with assessing the *observed* frequency of heavy rain event at the GYE station. 

The code below will read in 15-min observations, compute daily sums, and then evaluate the fraction of days (in our season of interest) that exceed the the relevant threshold. Multiplying this fraction by the number of days in the season gives us the expected seasonal frequency of flood events in the current, observed climate. 

Run the code below to read in the observed data and compute the necessary quantities. 

In [63]:
# Read in obs and format
obs=pd.read_excel("Data/15min.xlsx",index_col=0,parse_dates=True)
obs=obs[~obs.index.duplicated()]["P"]
new_index=pd.date_range(start=obs.index[0],end=obs.index[-1],freq="15Min")
obs=obs.reindex(new_index)
daysum=obs.resample('D').apply(lambda x: np.sum(x.values))
season=daysum.loc[daysum.index.month.isin(month_sel)]
nvalid=np.sum(~np.isnan(season.values[:])).astype(np.float)

# What is the observed probability of p>thresh
pref=np.sum(season.values[:]>thresh)/nvalid
print("Observed daily precipitation exceeded %.2f mm/d on %.2f%% of days, meaning flooding ~%.2f times/year"%\
      (thresh,pref*100,pref*ndays_year))

Observed daily precipitation exceeded 15.00 mm/d on 1.67% of days, meaning flooding ~3.05 times/year


## Downloading CMIP6 data from the Climate Data Store

We are now ready to download and process data from the CMIP6 archive. As explained earlier, we will work together in this to generate as many data points as we can. In your groups, you should obtain data from different **experiments** and **models**. The code below is setup to enter different options: "*model_name*" specifies which model to use; "*scenario_name*" indicates which experiment. The options you enter here will be used to generate a request from the Climate Data Store, and the data will be downloaded and processed automatically. 

To give the code the best chance of working without any glitches, you should ensure that valid options are entered. You can do this by accessing [this form](https://cds.climate.copernicus.eu/cdsapp#!/dataset/projections-cmip6?tab=form). **You must have logged in to your Copernicus account to use this form properly**. *[Go [here](https://cds.climate.copernicus.eu/user/register?destination=%2F%23!%2Fhome) to register if you did not complete this before now.]*

Once you are on the form, you can start to see which *models* have data by populating the form like so:

- Temporal resolution --> Daily

- Experiment --> Historical

- Level --> Single levels

- Variable --> Precipitation


The invalid experiment options for a given model (those that do not have precipitation data for the above combination) will be greyed out.  

From the remaining *valid* options, select a combination model and ssp-rcp experiment combination that has availability.  

Next, you need to check that the *temperature* data are also available for the historical/future experiments for this model. You can do this by setting the **Temporal resolution to Monthly** and **Variable to Near-surface air temperature**. If temperature data are also held in the archive, you should click **Show API request"** at the bottom of the form. 

The text that pops up tells us how we should format our "query to submit to the CDS server. Please take a note of how the "experiment" is written (i.e., the text after "experiment": ); and of the way the model is written (i.e., the text after "model": ). You must enter these options for *senario_name* and *model_name* in the code below -- **exactly** as they appear here. 

When you think you have understood the above, please try to run the code below. I say *try* because the server may fail on large requests -- even if you do everything 'right'. 

In [61]:
# * * * * * * * * * * * * * * * * * * * #
# Setting up
# * * * * * * * * * * * * * * * * * * * #
#---------------------------------------#

# *** !! CHANGE THE BELOW !! *** 
model_name="cesm2" # <== CHANGE ME * * * * 
scenario_name="ssp5_8_5" # <== CHANGE ME * * * * 
#---------------------------------------#

exps=["historical",scenario_name,"historical",scenario_name]
freq=["daily","daily","monthly","monthly"]
vs=["precipitation","precipitation",
   'near_surface_air_temperature','near_surface_air_temperature']
dates=['1995-01-01/2014-12-31','2021-01-01/2100-12-31',
       '1995-01-01/2014-12-31','2021-01-01/2100-12-31']
# * * * * * * * * * * * * * * * * * * * #

# * * * * * * * * * * * * * * * * * * * #
# Downloading
# * * * * * * * * * * * * * * * * * * * #
# Clear the storage folder of legacy netcdf
# files
[os.remove(folder_store+ii) for ii in os.listdir(folder_store) if ".nc" in ii]
for i in range(len(exps)):
    if i <2: area=['55','-3','50','3',]
    else: area=['90','-180','-90','180',]
    c.retrieve( # Start downloading
            'projections-cmip6',
            {
                'temporal_resolution': '%s'%freq[i],
                'experiment': '%s'%exps[i],
                'level': 'single_levels',
                'variable': '%s'%vs[i],
                'model': '%s'%model_name, 
                'date': '%s'%dates[i],
                'area': area,
                'format': 'zip',
            },
            folder_store+'out.zip',

    ) # End downloading   
    
    # ---------------------------------------#
    # Processing data 
    # ---------------------------------------#
    # Extract from a zip file 
    with zipfile.ZipFile(folder_store+'out.zip', 'r') as zip:
        zip.extractall(folder_store)
        
# Now process all the prfiles
files=[folder_store + ii for ii in os.listdir(folder_store) if "pr_day" in ii and ".nc" in ii]
din=xa.open_mfdataset(files)
nearest=din.sel(lat=lat_lough,lon=lon_lough,method="nearest")
pmod=nearest["pr"].sel\
(time=nearest.time.dt.month.isin(month_sel)).groupby('time.year').max().to_dataframe()*seconds_in_day
  
# Repeat, tas
files=[folder_store + ii for ii in os.listdir(folder_store) if "tas_Amon" in ii and ".nc" in ii]
din=xa.open_mfdataset(files)
t=din.tas-273.15
wts=np.cos(np.deg2rad(t.lat)); wts.name="weights"     
tmod=t.weighted(wts).mean(("lon","lat")).groupby('time.year').mean().to_dataframe("tas") 

# Tidy up        
_=[os.remove(folder_store+ii) for ii in folder_store if ".nc" in ii or "adaptor_" in ii]

print("\n\n* * * DOWNLOADS COMPLETE * * * \n")

2021-10-15 17:19:19,576 INFO Welcome to the CDS
2021-10-15 17:19:19,577 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/projections-cmip6
2021-10-15 17:19:19,627 INFO Request is completed
2021-10-15 17:19:19,627 INFO Downloading https://download-0010.copernicus-climate.eu/cache-compute-0010/cache/data4/adaptor.esgf_wps.retrieve-1634296415.3452716-11391-13-7170ab4e-e5f1-49ea-8f83-d57329137aa6.zip to /home/lunet/gytm3/Teaching/GYP050/Dev/CMIP6Data/out.zip (846.4K)
2021-10-15 17:19:19,785 INFO Download rate 5.3M/s
2021-10-15 17:19:19,813 INFO Welcome to the CDS
2021-10-15 17:19:19,814 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/projections-cmip6
2021-10-15 17:19:20,039 INFO Downloading https://download-0013.copernicus-climate.eu/cache-compute-0013/cache/data3/adaptor.esgf_wps.retrieve-1634296559.0051897-12235-14-259e3ce6-c080-4f3e-a227-f8477d1153f8.zip to /home/lunet/gytm3/Teaching/GYP050/Dev/CMIP6Data/out.zip (2.8M)
2021-10-15 17:



* * * DOWNLOADS COMPLETE * * * 



## Processing the CMIP6 data

If the downloads completed -- great! That's the trickiest part out the way. 

Within the download code there were a few 'tricks' embedded: we already processed the monthly air temperatures to global MAT, and we also computed RX1, using only the months of the season we're interested in (summer). That's most of the 'heavy lifting' done. It was achieved in a few lines of code using the wonderful [xarray](http://xarray.pydata.org/en/stable/) python module, and whilst the programming details are beyond the scope of this module, hope you can appreciate how little effort (code) it took to wrangle the data into the format we wanted. 

We are now ready to see the fruits of our labour. In the code below, the difference in mean RX1 between the historical experiment (1995-2014) and different future periods (each 20 years in duration) is computed as a *percentage* of the historical RX1 day. This percentage is used to scale up (or down) the *observed* precipitation. We then compute the frequency with with the flood threshold is passed in this *projected* climate. Scenarios are presented in the expected number of days per year, by computing the fraction of days exceeding the threshold, and multiplying this by the number of days per season (i.e., 183 in summer). Finally we also compute the difference (in $^{\circ}$C) between the future and historical periods. We add 0.68 $^{\circ}$C to this value to account for the fact that 1995-2014 was *already* this much warmer than pre-industrial. 

When you are ready, run the code below to generate the results. If everything works, you have enough data to populate some additional rows in the [shared spreadsheet](https://docs.google.com/spreadsheets/d/1SGONp3xEh62SLVh_fbkTSiarNls5ArcxXieAt51qvjw/edit?usp=sharing)!

In [64]:
period_st=[2021,2041,2081]
period_stop=[2040,2060,2100]
tbase=tmod["tas"].loc[tmod.index<2015].mean()
pbase=pmod["pr"].loc[pmod.index<2015].mean()
for i in range(len(period_st)):
    idx=np.logical_and(pmod.index>=period_st[i],pmod.index<=period_stop[i])
    pc=pmod["pr"].loc[idx].mean()/pbase
    tdif=out["tfuture"]["tas"].loc[idx].mean()-tbase+pid
    projected=season*pc
    pdays_year=np.sum(projected>thresh)/nvalid*ndays_year  
    print("For period: %.0f -> %.0f, RX1 changes by %.2f%% & MAT is %.2fC above pre-industrial. Flood days = %.2f/year"\
          %(period_st[i],period_stop[i],(pc-1)*100,tdif,pdays_year))

For period: 2021 -> 2040, RX1 changes by 3.66% & MAT is 1.26C above pre-industrial. Flood days = 3.65/year
For period: 2041 -> 2060, RX1 changes by 12.48% & MAT is 2.27C above pre-industrial. Flood days = 4.75/year
For period: 2081 -> 2100, RX1 changes by 34.81% & MAT is 5.04C above pre-industrial. Flood days = 7.80/year


If everything worked -- congratulations on processing the data successfully! You can now repeat the steps above **for a different moddel and/or experiment** (just change *model_name* and *experiment_name* and run the two code cells above -- in the same order as before). With each new model/experiment combination, we can populate a new row of our spreadsheet! 

## Going further

If you have the appetite for more, why not try starting again but running the analysis for **summer**. in combination with the winter analysis, this could lay the foundation for an interesting dissertation...
